In [1]:
import os
import torch
import glob
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pydicom
import os
import numpy as np
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import math
from torch.distributions.normal import Normal
import torch.cuda

In [2]:
def prep_dataframe(directory, csv):   
    diagnoses = pd.read_csv(csv)
    diagnoses = diagnoses.to_numpy()
    more = np.empty((len(diagnoses), 3), dtype = 'U18')
    for x in range(len(diagnoses)):
        more[x][0] = diagnoses[x][0][3:12]
        more[x][1] = diagnoses[x][0][13:]

    more[:,2] = diagnoses[:,1]

    #test_list = os.listdir(directory)
    test_list = glob.glob("/home/wustl/brooks.kennedy/stage_2_train/*.dcm")[0:100]
    #print(test_list)
    image_arrays = np.zeros([512, 512])
    df = pd.DataFrame(columns = ["Image", "ID", "Diagnoses"])

    for file in test_list:
        if(file != ".DS_Store"):
            try:
                dataset = pydicom.dcmread(file, force = True)
                diagnoses = more[:,1:][more[:,0]==file[44:53]]
                filtered_diagnoses = diagnoses[:,0][diagnoses[:,1]=='1']
                if filtered_diagnoses.size != 0:
                    df = df.append({"Image": dataset.pixel_array, "ID": file[44:53], 
                                    "Diagnoses": filtered_diagnoses}, ignore_index = True)
            except:
                print("Error with file "+file) 
    del diagnoses
    del more
    del dataset
    return df

In [3]:
train_data_ = prep_dataframe("/home/wustl/brooks.kennedy/stage_2_train", "/home/wustl/brooks.kennedy/stage_2_train.csv")

In [4]:
train_x = train_data_['Image']
ids = train_data_['ID']

#convert training images, target data, ids to numpy arrays
train_x = train_x.to_numpy()
ids = ids.to_numpy()
for x in range(0, len(train_x)): #filter out incorrectly shaped images
    if (train_x[x].shape != (512, 512)):
        train_x = np.delete(train_x, x)
        ids = np.delete(ids, x)
train_x = np.array([np.array(x) for x in train_x])

#convert training images to pytorch tensors
train_x = train_x.reshape(len(train_x), 1, 512, 512)
train_x = train_x.astype('float')
train_x = torch.from_numpy(train_x)

del train_data_

In [7]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()       
#         self.cnn_layers = Sequential(
#             #convolution layer 1
#             Conv2d(16, 4, kernel_size = 3, stride=1, padding=1),
#             BatchNorm2d(4),
#             ReLU(inplace=True),
#             MaxPool2d(kernel_size = 2, stride=2),
            
#             #convolution layer 2
#             Conv2d(4, 16, kernel_size = 3, stride=1, padding=1),
#             BatchNorm2d(16),
#             ReLU(inplace=True),
#             MaxPool2d(kernel_size = 2, stride=2),
#         )
        self.dense_layer = Sequential (
            nn.Linear(16, 128),
            nn.Linear(128, 256),
            nn.Linear(256, 262144)
        )
        self.relu = nn.ReLU()
        
    def forward(self, x):
        #x = self.cnn_layers(x)
        #x = self.cnn_layers(x)
        #torch.reshape(x, (4, 4))
        x = self.dense_layer(x)
        x = self.relu(x)
        x = torch.reshape(x, (512, 512))
        return x

In [8]:
def makeFakeData(generator, count, real_data):
    mu = real_data.mean()
    std = real_data.std()
    generated_images = torch.empty(size=(count,1,512,512)).to(torch.device(0))
    for i in range(count):
        noise = torch.normal(mu, std, size=(4096, 16, 4, 4)).to(torch.device(0))
        generated_image = gen(noise)
        generated_image = generated_image.reshape((1,1,512,512))
        generated_images[i] = generated_image #torch.cat((generated_images, generated_image), 0)
    return generated_images


In [9]:
channels = 1
im_size = 512
noise_size = 16
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(channels, im_size, 4, 2, 1, bias = False),
            #BatchNorm2d(im_size),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            nn.Conv2d(im_size, im_size*2, 4, 2, 1, bias = False),
            nn.BatchNorm2d(im_size*2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            nn.Conv2d(im_size*2, im_size*4, 4, 2, 1, bias = False),
            nn.BatchNorm2d(im_size*4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            nn.Conv2d(im_size*4, im_size*8, 4, 2, 1, bias = False),
            nn.BatchNorm2d(im_size*8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(im_size*8, 1, 4, 2, 1),
            nn.Sigmoid()
            
        )

#         self.linear_layers = Sequential(
#             Linear(256*256, 10)
#         )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        #x = x.view(x.size(0), -1)
        #x = self.linear_layers(x)
        return x

In [10]:
channels = 1
ngf = 512
gen_input_size = 16
ngpu = 1
class Generator2(nn.Module):
    
    def __init__(self, ngpu):
        super(Generator2, self).__init__()
        self.ngpu = ngpu
        self.layers = nn.Sequential(
                # input is Z, going into a convolution
                nn.ConvTranspose2d(gen_input_size, ngf * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 8),
                nn.ReLU(True),
                # state size. (ngf*8) x 4 x 4
                nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                # state size. (ngf*4) x 8 x 8
                nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                # state size. (ngf*2) x 16 x 16
                nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                # state size. (ngf) x 32 x 32
                nn.ConvTranspose2d( ngf, channels, 4, 2, 1, bias=False),
                nn.Tanh()
                # state size. (nc) x 64 x 64
            )

    def forward(self, x): 
        x = self.layers(x)
        return x

In [11]:
train_x = train_x.to(torch.device(0))
gen = Generator2(ngpu).to(torch.device(0))
im = makeFakeData(gen, 1, train_x[0:100])
# dis = Discriminator().to(torch.device(0))
# discrim = dis(im)

RuntimeError: CUDA out of memory. Tried to allocate 6.12 GiB (GPU 0; 16.00 GiB total capacity; 12.94 GiB already allocated; 832.80 MiB free; 12.96 GiB reserved in total by PyTorch)

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.device("cuda")

In [ ]:
import gc
gc.collect()